In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from reactit.reactit import ReactionsListGenerator
from reactit.reactit import MappingtoReaction

import tqdm
compounds = {0:'CO2',
              1:'H2O', 
              2:'SO2', 
              3:'NO2', 
              4:'H2S', 
              5:'CH4', 
              6:'CH3', 
              7:'C2H4',
              8:'N2',
              9:'O2',
              10:'H2',
              11:'CO',
              12:'S8',
              13:'H2SO4',
              14:'H2O2',
              15:'HNO3',
              16:'H2SO3',
              17:'HNO2',
              18:'C2H4',
              19:'NH4',
              20:'NH3',
              21:'SO3',
              22:'O3',
              23:'C2H6',
              24:'C3H8',
              25:'C2H5OH',
              26:'HCHO',
              27:'CH3COOH',
              28:'C4H10',
              29:'CH3NH2',
              30:'CH3CONH2'}
#compounds = {0:'O2',1:'H2O',2:'H2'}

In [4]:
rlg = ReactionsListGenerator(no_compounds=len(compounds))
reactions = list(rlg.iterate(4))

In [5]:
mappings = MappingtoReaction(reactions = reactions, compounds = compounds)
final_reactions = mappings.run_all()

100%|██████████| 12257/12257 [00:25<00:00, 477.99it/s]


In [6]:
import pandas as pd 
pd.Series(final_reactions)

0                    1 CH4 + 1 C3H8 = 4 CH3
1                     6 CH3 = 1 H2 + 2 C3H8
2                   1 H2 + 3 C4H10 = 4 C3H8
3                  2 CH3 + 1 C2H4 = 1 C4H10
4                   2 CH4 + 1 C2H4 = 2 C2H6
                       ...                 
1982     1 NH3 + 1 C3H8 = 1 C2H6 + 1 CH3NH2
1983        1 H2O + 1 C3H8 = 2 CH4 + 1 HCHO
1984     1 H2SO3 + 2 C2H4 = 1 SO3 + 1 C4H10
1985        1 O2 + 1 C2H6 = 1 H2O2 + 1 C2H4
1986    1 O2 + 1 C4H10 = 1 C2H6 + 1 CH3COOH
Length: 1987, dtype: object

In [7]:
def mp_function(iterable):
    reactants = {molecule:mappings.parse_molecule(molecule) for molecule in iterable[0]}
    products = {molecule:mappings.parse_molecule(molecule) for molecule in iterable[1]}
    balanced = mappings.balance_reaction(reactants,products)
    if balanced:
        return(balanced)
    #else:
    #    return(None)
    
import tqdm_pathos 

rlg = ReactionsListGenerator(no_compounds=len(compounds))
reactions = list(rlg.iterate(4))
mappings = MappingtoReaction(reactions = reactions, compounds = compounds)
approved = mappings.remove_indices(reactions)
strings = mappings.convert_to_string(approved)

screened = tqdm_pathos.map(mp_function,strings)
screened = [x for x in screened if x is not None]

  0%|          | 0/12257 [00:00<?, ?it/s]

In [8]:
import pandas as pd 
pd.Series(screened)

0                    1 CH4 + 1 C3H8 = 4 CH3
1                     6 CH3 = 1 H2 + 2 C3H8
2                   1 H2 + 3 C4H10 = 4 C3H8
3                  2 CH3 + 1 C2H4 = 1 C4H10
4                   2 CH4 + 1 C2H4 = 2 C2H6
                       ...                 
1982     1 NH3 + 1 C3H8 = 1 C2H6 + 1 CH3NH2
1983        1 H2O + 1 C3H8 = 2 CH4 + 1 HCHO
1984     1 H2SO3 + 2 C2H4 = 1 SO3 + 1 C4H10
1985        1 O2 + 1 C2H6 = 1 H2O2 + 1 C2H4
1986    1 O2 + 1 C4H10 = 1 C2H6 + 1 CH3COOH
Length: 1987, dtype: object